In [2]:
from __future__ import print_function
import torch as t

In [4]:
#在创建tensor的时候指定requires_grad
a = t.randn(3,4, requires_grad=True)
# 或者
a = t.randn(3,4).requires_grad_()
# 或者
a = t.randn(3,4)
a.requires_grad = True
a

tensor([[ 0.1953,  0.0306, -0.0437, -1.0927],
        [-0.0554, -0.7856, -0.2682, -0.7996],
        [-0.9014, -0.2513, -0.3172, -0.3856]], requires_grad=True)

In [5]:
b = t.zeros(3,4).requires_grad_()
b

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], requires_grad=True)

In [6]:
# 也可写成c = a + b
c = a.add(b)
c

tensor([[ 0.1953,  0.0306, -0.0437, -1.0927],
        [-0.0554, -0.7856, -0.2682, -0.7996],
        [-0.9014, -0.2513, -0.3172, -0.3856]], grad_fn=<AddBackward0>)

In [7]:
d = c.sum()
d.backward() # 反向传播

In [9]:
d.requires_grad

True

In [11]:
a.grad

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [13]:
b.grad

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [14]:
a.is_leaf, b.is_leaf, c.is_leaf

(True, True, False)

In [15]:
c.grad is None

True

计算下面这个函数的导函数：
$$
y = x^2\bullet e^x
$$
它的导函数是：
$$
{dy \over dx} = 2x\bullet e^x + x^2 \bullet e^x
$$
来看看autograd的计算结果与手动求导计算结果的误差。

In [29]:
def f(x):
    y = x **2 * t.exp(x)
    return y

def gradf(x):
    dx = 2*x*t.exp(x) + x**2 *t.exp(x)
    return dx

In [30]:
x = t.randn (3,4, requires_grad=True)
y = f(x)
y

tensor([[0.0302, 0.4501, 0.9540, 0.0776],
        [0.1023, 0.1190, 0.6829, 0.2627],
        [0.1443, 0.0138, 0.0762, 3.3339]], grad_fn=<MulBackward0>)

In [31]:
y.backward(t.ones(y.size()))
x.grad

tensor([[-0.2856, -0.2648,  3.7140, -0.3952],
        [ 0.8373, -0.4383,  2.9242, -0.4442],
        [-0.4522, -0.2071,  0.7001,  9.5679]])

In [32]:
dx = gradf(x)
dx

tensor([[-0.2856, -0.2648,  3.7140, -0.3952],
        [ 0.8373, -0.4383,  2.9242, -0.4442],
        [-0.4522, -0.2071,  0.7001,  9.5679]], grad_fn=<AddBackward0>)